In [1]:
import pandas as pd
import numpy as np

from influxdb_client import InfluxDBClient, Point, Dialect

import re
import time
import datetime

import warnings
from influxdb_client.client.warnings import MissingPivotFunction

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as colors

import pandasql as ps
import sqlite3

import csv

import warnings 
from influxdb_client.client.warnings import MissingPivotFunction
warnings.simplefilter("ignore", MissingPivotFunction)

import gc

pd.set_option('display.max_rows', 500)

In [2]:
def _parse_line(line):

    rx_dict = {
    'token': re.compile(r'var token = "(?P<token>.*)"\n'),
    'url': re.compile(r'var url = "(?P<url>.*)"\n'),
    'org': re.compile(r'var org = "(?P<org>.*)"\n'),
    'bucket': re.compile(r'var bucket = "(?P<bucket>.*)"\n'),
    }   

    """
    Do a regex search against all defined regexes and
    return the key and match result of the first matching regex

    """
    for key, rx in rx_dict.items():
        match = rx.search(line)
        if match:
            return key, match
    # if there are no matches
    return None, None

filepath = '/root/flexi-pipe/config.go'
# open the file and read through it line by line
with open(filepath, 'r') as file_object:
    line = file_object.readline()
    while line:
        # at each line check for a match with a regex
        key, match = _parse_line(line)

        if key == 'token':
            token = match.group('token')
        elif key == 'url':
            url = match.group('url')
        elif key == 'org':
            org = match.group('org')
        elif key == 'bucket':
            bucket = match.group('bucket')
        
        line = file_object.readline()
# url="http://192.168.20.58:8086"
url = "http://localhost:8086"

In [3]:
start = 1692978196
end = 1694744007

In [4]:
def experiment(start_time, end_time, filepath):
    # Retrieve experiments data from csv
    data = pd.read_csv(filepath, header=None)
    df = pd.DataFrame(data)

    #Rename columns
    experiments = df.rename(columns={0: "start", 1: "end", 2: "topology", 3: "runtime", 4: "parameter", 5: "d", 6: "dlo", 7: "dhi", 8: "dscore", 9: "dlazy", 10: "dout", 11: "gossipFactor", 12: "initialDelay", 13: "interval"}, errors='raise')

    #Correct timestamp
    experiments["start"] = experiments["start"].str.slice(0, 27)
    experiments["end"] = experiments["end"].str.slice(0, 27)

    #String to timestamp
    # experiments['startUnix'] = pd.to_datetime(experiments["start"],format="%Y-%m-%d %H:%M:%S.%f").astype('int64') / 10**9
    # experiments['endUnix'] = pd.to_datetime(experiments["end"],format="%Y-%m-%d %H:%M:%S.%f").astype('int64') / 10**9
    experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9
    experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9


    experiments['startUnix'] = pd.to_timedelta(experiments['startUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)
    experiments['endUnix'] = pd.to_timedelta(experiments['endUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)

    #Drop fields we don't mneed for the moment
    exp = experiments.drop(columns=["runtime", "initialDelay"]).sort_values(by=["start"])

    #Get times for different intervals
    # intervals = exp["interval"].drop_duplicates().sort_values().reset_index(drop=True)
    # intervals.head(10)

    expTime = exp[exp['startUnix'].astype(int).between(start_time, end_time)]
    # expTime['experiment'] = expTime.index
    expTime = expTime.reset_index().rename({'index':'experiment'}, axis = 'columns')

    return expTime

experiments = experiment(start, end, '../experiments.csv')
experiments.head(5)

/tmp/ipykernel_38368/1055150111.py:16: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9
/tmp/ipykernel_38368/1055150111.py:17: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed", infer_datetime_format=True).astype('int64') / 10**9


,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
0,150,2023-08-25 15:43:16.3243323,2023-08-25 16:13:16.3255309,unl,reference,8,6,12,4,8,2,0.25,1.0,1692978196,1692979996
1,151,2023-08-25 16:16:49.6961810,2023-08-25 16:46:49.6974978,unl,reference,8,6,12,4,8,2,0.25,1.0,1692980209,1692982009
2,152,2023-08-25 16:50:23.6899992,2023-08-25 17:20:23.6921991,unl,reference,8,6,12,4,8,2,0.25,1.0,1692982223,1692984023
3,153,2023-08-25 17:23:57.7272734,2023-08-25 17:53:57.8279711,unl,interval,8,6,12,4,8,2,0.25,0.5,1692984237,1692986037
4,154,2023-08-25 17:57:30.6350173,2023-08-25 18:27:30.7363305,unl,interval,8,6,12,4,8,2,0.25,0.5,1692986250,1692988050


In [5]:
def from_influx(url, token, org, measurement, start_time, end_time,grouping_key):
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)

    # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()

    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                                        ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                                        ' |> filter(fn: (r) => r._measurement == "'+measurement+'") '
                                        ' |> group(columns: ["_measurement", "_field"], mode: "by") '
                                        ' |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')
    client.close()

    # df = data_frame.drop(columns=['result', 'table','_start', '_stop', '_measurement', 'topic', 'receivedFrom']).sort_values(by=["_time"]).reset_index(drop=True)
    data_frame.reset_index(inplace=True)
    df = data_frame[['_time', grouping_key]].sort_values(by=["_time"]).reset_index(drop=True)
    df["_time"] = pd.to_datetime(df["_time"])

    return df

In [6]:
def from_influx_count(url, token, org, start_time, end_time,grouping_key):
    # start_time = 1693222601 
    # end_time = 1693228430    
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)
        
        # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()
        
    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                       ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                        '|> filter(fn: (r) => r._measurement == "deliverMessage") '
                        '|> group(columns: ["_measurement", "_field"], mode: "by") '
                        '|> count()')
    client.close()

    if data_frame.empty:
        count = 0
    else:
        count = data_frame["_value"].min().astype(int)
    return count

In [7]:
#Validate data
validate = pd.DataFrame()

for index, row in experiments.iterrows():
    count = from_influx_count(url, token, org, row["startUnix"], row["endUnix"],"_measurement")
    d = {'experiment': [row['experiment']], 'count': [count], 'topology': [row["topology"]], 'd': [row["d"]],'dhi': [row["dhi"]],'dlo': [row["dlo"]],'dlazy': [row["dlazy"]],
        'dscore': [row["dscore"]],'dout': [row["dout"]],'gossipFactor': [row["gossipFactor"]],'interval': [row["interval"]],}
    aux = pd.DataFrame(data=d)
    validate = pd.concat([validate, aux])

# validate.head(200)

exps = experiments.merge(validate, on=['experiment', 'topology','d','dhi','dlo','dlazy','dscore','dout','gossipFactor','interval'])
exps = exps.loc[exps["count"]>0]
# exps.to_csv('exp_filtered.csv')
exps.head(10)

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix,count
2,152,2023-08-25 16:50:23.6899992,2023-08-25 17:20:23.6921991,unl,reference,8,6,12,4,8,2,0.25,1.0,1692982223,1692984023,17416
3,153,2023-08-25 17:23:57.7272734,2023-08-25 17:53:57.8279711,unl,interval,8,6,12,4,8,2,0.25,0.5,1692984237,1692986037,75673
4,154,2023-08-25 17:57:30.6350173,2023-08-25 18:27:30.7363305,unl,interval,8,6,12,4,8,2,0.25,0.5,1692986250,1692988050,153137
5,155,2023-08-25 18:31:03.6332371,2023-08-25 19:01:03.6935448,unl,interval,8,6,12,4,8,2,0.25,0.5,1692988263,1692990063,125112
6,156,2023-08-25 19:04:36.9266682,2023-08-25 19:34:36.9337235,unl,interval,8,6,12,4,8,2,0.25,30.0,1692990276,1692992076,215731
7,157,2023-08-25 19:38:10.1780791,2023-08-25 20:08:10.1796217,unl,interval,8,6,12,4,8,2,0.25,30.0,1692992290,1692994090,274279
8,158,2023-08-25 20:11:43.6270210,2023-08-25 20:41:43.6285805,unl,interval,8,6,12,4,8,2,0.25,30.0,1692994303,1692996103,232539
9,159,2023-08-25 20:45:16.8407778,2023-08-25 21:15:16.8427550,unl,interval,8,6,12,4,8,2,0.25,3.0,1692996316,1692998116,172720
10,160,2023-08-25 21:18:50.1678891,2023-08-25 21:48:50.1692797,unl,interval,8,6,12,4,8,2,0.25,3.0,1692998330,1693000130,195350
11,161,2023-08-25 21:52:23.8902604,2023-08-25 22:22:23.8915283,unl,interval,8,6,12,4,8,2,0.25,3.0,1693000343,1693002143,171780


In [8]:
#Remove fault executions

#get median
median = exps.drop(columns=['start','end','startUnix','endUnix', 'parameter', 'experiment']).drop_duplicates()
median = median.groupby(['topology','d','dhi','dlo','dlazy','dscore','dout','gossipFactor','interval']).agg({'count':['median', 'std', 'mean']})
median.columns = median.columns.droplevel(0)
median.reset_index(inplace=True)
median = median.loc[median['median'] > 0]

median.head(50)

,topology,d,dhi,dlo,dlazy,dscore,dout,gossipFactor,interval,median,std,mean
0,general,6,12,6,8,4,2,0.25,1.00,105329.0,15050.640263,100257.333333
1,general,8,6,6,8,4,2,0.25,1.00,70111.0,29335.386589,73581.666667
2,general,8,8,6,8,4,2,0.25,1.00,131030.0,13185.434249,126814.666667
3,general,8,12,2,8,4,2,0.25,1.00,95746.0,31170.399436,85077.000000
4,general,8,12,3,8,4,2,0.25,1.00,87312.0,46236.179113,84179.000000
5,general,8,12,6,4,4,2,0.25,1.00,91655.0,24982.929419,100854.666667
6,general,8,12,6,6,4,2,0.25,1.00,114225.0,27813.669307,99161.333333
7,general,8,12,6,8,2,2,0.25,1.00,111506.0,13659.075896,109774.333333
8,general,8,12,6,8,4,1,0.25,1.00,116063.0,8118.918052,116818.666667
9,general,8,12,6,8,4,2,0.25,0.50,74038.0,31175.417003,66333.333333


In [9]:
#Validate the data

validation = exps.merge(median, on=['topology','d','dhi','dlo','dlazy','dscore','dout','gossipFactor','interval'])
validated = validation[(validation['count'] >= validation['mean']-(0.15*validation['std']))]
validated = validated.drop(columns=['startUnix', 'endUnix'])
validated.dropna()

validated.head(100)

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,count,median,std,mean
1,300,2023-08-31 17:06:48.9141201,2023-08-31 17:36:48.9166680,unl,reference,8,6,12,4,8,2,0.25,1.00,128481,151324.5,103718.978020,117825.666667
6,328,2023-09-12 18:00:05.6473282,2023-09-12 18:30:05.6489429,unl,reference,8,6,12,4,8,2,0.25,1.00,194713,151324.5,103718.978020,117825.666667
7,329,2023-09-12 18:33:30.3361846,2023-09-12 19:03:30.3400825,unl,reference,8,6,12,4,8,2,0.25,1.00,243619,151324.5,103718.978020,117825.666667
8,330,2023-09-12 19:07:00.3403636,2023-09-12 19:37:00.3500787,unl,reference,8,6,12,4,8,2,0.25,1.00,199084,151324.5,103718.978020,117825.666667
9,331,2023-09-12 19:40:30.1978792,2023-09-12 20:10:30.2017551,unl,reference,8,6,12,4,8,2,0.25,1.00,208266,151324.5,103718.978020,117825.666667
10,332,2023-09-12 20:14:00.4860767,2023-09-12 20:44:00.4880682,unl,reference,8,6,12,4,8,2,0.25,1.00,174168,151324.5,103718.978020,117825.666667
11,333,2023-09-12 20:47:31.5355445,2023-09-12 21:17:31.5383218,unl,reference,8,6,12,4,8,2,0.25,1.00,239737,151324.5,103718.978020,117825.666667
13,154,2023-08-25 17:57:30.6350173,2023-08-25 18:27:30.7363305,unl,interval,8,6,12,4,8,2,0.25,0.50,153137,125112.0,39222.201710,117974.000000
14,155,2023-08-25 18:31:03.6332371,2023-08-25 19:01:03.6935448,unl,interval,8,6,12,4,8,2,0.25,0.50,125112,125112.0,39222.201710,117974.000000
16,157,2023-08-25 19:38:10.1780791,2023-08-25 20:08:10.1796217,unl,interval,8,6,12,4,8,2,0.25,30.00,274279,232539.0,30145.770206,240849.666667


In [10]:
final = validated.drop(columns=['experiment', 'start','end', 'parameter', 'count', 'std', 'median', 'mean']).drop_duplicates()
final['n_nodes'] = 23
final = final.reset_index().rename({'index':'identifier'}, axis = 'columns')
final.head(10)

,identifier,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,n_nodes
0,1,unl,8,6,12,4,8,2,0.25,1.0,23
1,13,unl,8,6,12,4,8,2,0.25,0.5,23
2,16,unl,8,6,12,4,8,2,0.25,30.0,23
3,19,unl,8,6,12,4,8,2,0.25,3.0,23
4,22,unl,24,6,12,4,8,2,0.25,1.0,23
5,25,unl,12,6,12,4,8,2,0.25,1.0,23
6,27,unl,6,6,12,4,8,2,0.25,1.0,23
7,30,unl,8,2,12,4,8,2,0.25,1.0,23
8,33,unl,8,3,12,4,8,2,0.25,1.0,23
9,38,unl,8,8,12,4,8,2,0.25,1.0,23


In [11]:
#Limit extraction to 15 minutes

experiments = validated.drop(columns=['count','median', 'mean','std']).rename(columns={'start':'originalStart','end':'originalEnd'})

experiments['originalStart'] = pd.to_datetime(experiments["originalStart"], format='mixed')
experiments['originalEnd'] = pd.to_datetime(experiments["originalEnd"], format='mixed')

experiments['start'] = experiments['originalStart'] + pd.Timedelta(hours=0, minutes=7, seconds=30)
experiments['end'] = experiments['originalEnd'] - pd.Timedelta(hours=0, minutes=7, seconds=30)

experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed").astype('int64') / 10**9
experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed").astype('int64') / 10**9

experiments['startUnix'] = pd.to_timedelta(experiments['startUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)
experiments['endUnix'] = pd.to_timedelta(experiments['endUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)

experiments.head(10)

,experiment,originalStart,originalEnd,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,start,end,startUnix,endUnix
1,300,2023-08-31 17:06:48.914120100,2023-08-31 17:36:48.916668000,unl,reference,8,6,12,4,8,2,0.25,1.0,2023-08-31 17:14:18.914120100,2023-08-31 17:29:18.916668000,1693502058,1693502958
6,328,2023-09-12 18:00:05.647328200,2023-09-12 18:30:05.648942900,unl,reference,8,6,12,4,8,2,0.25,1.0,2023-09-12 18:07:35.647328200,2023-09-12 18:22:35.648942900,1694542055,1694542955
7,329,2023-09-12 18:33:30.336184600,2023-09-12 19:03:30.340082500,unl,reference,8,6,12,4,8,2,0.25,1.0,2023-09-12 18:41:00.336184600,2023-09-12 18:56:00.340082500,1694544060,1694544960
8,330,2023-09-12 19:07:00.340363600,2023-09-12 19:37:00.350078700,unl,reference,8,6,12,4,8,2,0.25,1.0,2023-09-12 19:14:30.340363600,2023-09-12 19:29:30.350078700,1694546070,1694546970
9,331,2023-09-12 19:40:30.197879200,2023-09-12 20:10:30.201755100,unl,reference,8,6,12,4,8,2,0.25,1.0,2023-09-12 19:48:00.197879200,2023-09-12 20:03:00.201755100,1694548080,1694548980
10,332,2023-09-12 20:14:00.486076700,2023-09-12 20:44:00.488068200,unl,reference,8,6,12,4,8,2,0.25,1.0,2023-09-12 20:21:30.486076700,2023-09-12 20:36:30.488068200,1694550090,1694550990
11,333,2023-09-12 20:47:31.535544500,2023-09-12 21:17:31.538321800,unl,reference,8,6,12,4,8,2,0.25,1.0,2023-09-12 20:55:01.535544500,2023-09-12 21:10:01.538321800,1694552101,1694553001
13,154,2023-08-25 17:57:30.635017300,2023-08-25 18:27:30.736330500,unl,interval,8,6,12,4,8,2,0.25,0.5,2023-08-25 18:05:00.635017300,2023-08-25 18:20:00.736330500,1692986700,1692987600
14,155,2023-08-25 18:31:03.633237100,2023-08-25 19:01:03.693544800,unl,interval,8,6,12,4,8,2,0.25,0.5,2023-08-25 18:38:33.633237100,2023-08-25 18:53:33.693544800,1692988713,1692989613
16,157,2023-08-25 19:38:10.178079100,2023-08-25 20:08:10.179621700,unl,interval,8,6,12,4,8,2,0.25,30.0,2023-08-25 19:45:40.178079100,2023-08-25 20:00:40.179621700,1692992740,1692993640


In [12]:
def from_influx_message(url, token, org, start_time, end_time):
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)

    # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()

    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                                        ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                                        ' |> filter(fn: (r) => r._measurement == "message" and r._field == "type" and (r._value == 11 or r._value == 12 or r._value == 3 or r._value == 2)) '
                                        ' |> group(columns: ["_measurement", "_field"], mode: "by") '
                                        ' |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')
    client.close()

    # df = data_frame.drop(columns=['result', 'table','_start', '_stop', '_measurement', 'topic', 'receivedFrom']).sort_values(by=["_time"]).reset_index(drop=True)
    data_frame.reset_index(inplace=True)
    df = data_frame[['_time', 'type']].sort_values(by=["_time"]).reset_index(drop=True)
    df["_time"] = pd.to_datetime(df["_time"])

    return df

In [ ]:
message = pd.DataFrame()
execs = experiments.drop(columns=['originalStart', 'originalEnd'])
# final=final.loc[final["identifier"] == 1]

for index, row in final.iterrows():
    # print(row["start"], row["end"], row["experiment"])
    execs = exps.loc[(exps["topology"] == row["topology"]) & (exps["d"] == row["d"]) &
                            (exps["dlo"] == row["dlo"]) & (exps["dhi"] == row["dhi"]) &
                            (exps["dscore"] == row["dscore"]) & (exps["dlazy"] == row["dlazy"]) &
                            (exps["dout"] == row["dout"]) & (exps["gossipFactor"] == row["gossipFactor"]) &                    
                            (exps["interval"] == row["interval"])]

    for idx, exec in execs.iterrows():
        # print(exec["startUnix"], exec["endUnix"])
        query_mess = from_influx_message(url, token, org, exec["startUnix"], exec["endUnix"])

        query_mess["identifier"] = row["identifier"]

        query_mess["experiment"] = exec["experiment"]

        query_mess["start"] = exec["start"]
        query_mess["end"] = exec["end"]
            
        query_mess["_time"] = pd.to_datetime(query_mess["_time"])
        message = pd.concat([message,query_mess])

        query_mess.to_csv('./mess_'+str(exec["experiment"])+'.csv')
        
        del query_mess
        # del deliverMessage
        gc.collect()

message.head(10)

In [ ]:
message.to_csv('./messageTimeSeries.csv')

In [ ]:
del message
gc.collect()

In [ ]:
def from_influx(url, token, org, measurement, start_time, end_time,grouping_key):
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)

    # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()

    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                                        ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                                        ' |> filter(fn: (r) => r._measurement == "'+measurement+'") '
                                        ' |> group(columns: ["_measurement", "_field"], mode: "by") '
                                        ' |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')
    client.close()

    # df = data_frame.drop(columns=['result', 'table','_start', '_stop', '_measurement', 'topic', 'receivedFrom']).sort_values(by=["_time"]).reset_index(drop=True)
    data_frame.reset_index(inplace=True)
    df = data_frame[['_time', grouping_key]].sort_values(by=["_time"]).reset_index(drop=True)
    df["_time"] = pd.to_datetime(df["_time"])

    return df

In [ ]:
gossip = pd.DataFrame()
execs = experiments.drop(columns=['originalStart', 'originalEnd'])

for index, row in final.iterrows():
    # print(row["start"], row["end"], row["experiment"])
    execs = exps.loc[(exps["topology"] == row["topology"]) & (exps["d"] == row["d"]) &
                            (exps["dlo"] == row["dlo"]) & (exps["dhi"] == row["dhi"]) &
                            (exps["dscore"] == row["dscore"]) & (exps["dlazy"] == row["dlazy"]) &
                            (exps["dout"] == row["dout"]) & (exps["gossipFactor"] == row["gossipFactor"]) &                    
                            (exps["interval"] == row["interval"])]

    for idx, exec in execs.iterrows():
        # print(exec["startUnix"], exec["endUnix"])
        query_iwant = from_influx(url, token, org, "iwant", exec["startUnix"], exec["endUnix"],"_measurement")
        query_ihave = from_influx(url, token, org, "ihave", exec["startUnix"], exec["endUnix"],"_measurement")

        query_ihave["identifier"] = row["identifier"]
        query_iwant["identifier"] = row["identifier"]

        query_ihave["experiment"] = exec["experiment"]
        query_iwant["experiment"] = exec["experiment"]

        # query_mess["start"] = exec["start"]
        # query_mess["end"] = exec["end"]
            
        query_ihave["_time"] = pd.to_datetime(query_ihave["_time"])
        query_iwant["_time"] = pd.to_datetime(query_iwant["_time"])
        gossip = pd.concat([gossip, query_ihave, query_iwant])

        query_mess.to_csv('./gossip_'+str(exec["experiment"])+'.csv')
        
        del query_ihave
        del query_iwant
        # del deliverMessage
        gc.collect()

message.head(10)

In [ ]:
messageOverhead.to_csv('./gossipTimeSeries.csv')